# Dependencias

In [1]:
from IPython.display import clear_output
import os
import sys

import pandas as pd
import requests
import geopandas as gpd
from shapely import wkt, Point
import pyproj


In [2]:
!pip install basedosdados==2.0.0b16
clear_output()

# Constantes

In [7]:
# Camadas de Distritos extraida do GEOSAMPA pelo projeto SLUI
url_slui_dev = 'http://geoserver.slui.dev/geoserver/slui/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=slui%3Adistrito_municipal&maxFeatures=100&outputFormat=CSV'

billing_project_id = "br-fbsp-absp"

# Query para pegar os colegios do datalake BD
query = """
with dados_2023 as (
  SELECT
    *
FROM `basedosdados.br_inep_censo_escolar.escola`
where ano=2023 and sigla_uf="SP" and id_municipio="3550308"
),
localiza as (
  SELECT
    *
FROM `basedosdados.br_bd_diretorios_brasil.escola`
), tabela_log_lat as (
  select ano, dados_2023.sigla_uf, dados_2023.id_municipio, dados_2023.id_escola, endereco,
  quantidade_matricula_educacao_basica,
  quantidade_matricula_infantil,
  quantidade_matricula_infantil_creche,
  quantidade_matricula_infantil_pre_escola,
  quantidade_matricula_fundamental,
  quantidade_matricula_medio,
  quantidade_matricula_idade_0_3,
  quantidade_matricula_idade_4_5,
  quantidade_matricula_idade_6_10,
  quantidade_matricula_idade_11_14,
  quantidade_matricula_idade_15_17,
  quantidade_matricula_idade_18,
  safe_cast(latitude as float64) as latitude,
  safe_cast(longitude as float64) as longitude,
  from dados_2023
  left join localiza ON dados_2023.id_escola = localiza.id_escola
)

select *, ST_GEOGPOINT(longitude, latitude) as point from tabela_log_lat
"""

# Codigos

## Formar Tabela das escolas do Bigquery BD

In [8]:
import basedosdados as bd

df_escolas = bd.read_sql(
      query,
      billing_project_id=billing_project_id,
  )

Downloading: 100%|██████████|


## Transformar em GeoPandas

In [9]:
# Fazer GeoDataFrame dos distritos
distrito_municipal = pd.read_csv(url_slui_dev) # Pegar CSV
distrito_municipal['geometry'] = distrito_municipal['geometry'].apply(wkt.loads)
geopd_distrito_municipal = gpd.GeoDataFrame(distrito_municipal, geometry='geometry', crs="EPSG:31983")

# Transformar de EPSG:31983 para EPSG:4326
target_crs = pyproj.CRS.from_epsg(4326)
geopd_distrito_municipal = geopd_distrito_municipal.to_crs(target_crs)


# Transformar Tabela Escola para o um GeoDataFrame
df_escolas.point = df_escolas.point.apply(wkt.loads)
geo_df_escolas = gpd.GeoDataFrame(df_escolas, geometry='point', crs="EPSG:4326")
# geo_df_escolas.point = gdf_point

## Left join entre os distritos e as escolas

In [10]:
escolas_distritos = geopd_distrito_municipal.sjoin(geo_df_escolas, how='left', predicate='contains')
escolas_distritos.to_csv('escolas_distritos_geometry.csv', index=False, encoding='utf-8')
escolas_distritos.to_parquet('escolas_distritos_geometry.parquet', index=False)